In [375]:
import pandas as pd
import numpy as np

In [376]:
df = pd.read_csv("combined_file.csv")

In [377]:
df.head()

,Movie_id,Movie_name,Release_date,Overview,Ratings,Original_language,Top_5_cast,Directors,Genre_names
0,11224,Cinderella,1950-02-22,Cinderella has faith her dreams of a better li...,7.0,en,"Ilene Woods, Eleanor Audley, Verna Felton, Cla...","Hamilton Luske, Wilfred Jackson, Clyde Geronimi","Family, Fantasy, Animation, Romance"
1,599,Sunset Boulevard,1950-08-10,A hack screenwriter writes a screenplay for a ...,8.4,en,"William Holden, Gloria Swanson, Erich von Stro...",Billy Wilder,Drama
2,548,Rashomon,1950-08-26,Brimming with action while incisively examinin...,8.1,ja,"Toshirō Mifune, Machiko Kyō, Takashi Shimura, ...",Akira Kurosawa,"Crime, Drama, Mystery"
3,705,All About Eve,1950-10-06,From the moment she glimpses her idol at the s...,8.1,en,"Bette Davis, Anne Baxter, George Sanders, Cele...",Joseph L. Mankiewicz,Drama
4,37292,Broken Arrow,1950-08-01,Indian scout Tom Jeffords is sent out to stem ...,6.7,en,"James Stewart, Jeff Chandler, Debra Paget, Bas...",Delmer Daves,Western


In [378]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74000 entries, 0 to 73999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Movie_id           74000 non-null  int64  
 1   Movie_name         74000 non-null  object 
 2   Release_date       74000 non-null  object 
 3   Overview           70113 non-null  object 
 4   Ratings            74000 non-null  float64
 5   Original_language  74000 non-null  object 
 6   Top_5_cast         72463 non-null  object 
 7   Directors          73278 non-null  object 
 8   Genre_names        70334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 5.1+ MB


In [379]:
for i in df.columns:
    df[i] = df[i].replace(np.nan,"Unkown")

In [380]:
def remove_commas(text):
    cleaned_text = text.replace(',', ' ')  # Remove commas
    return cleaned_text

In [381]:
df.head()

,Movie_id,Movie_name,Release_date,Overview,Ratings,Original_language,Top_5_cast,Directors,Genre_names
0,11224,Cinderella,1950-02-22,Cinderella has faith her dreams of a better li...,7.0,en,"Ilene Woods, Eleanor Audley, Verna Felton, Cla...","Hamilton Luske, Wilfred Jackson, Clyde Geronimi","Family, Fantasy, Animation, Romance"
1,599,Sunset Boulevard,1950-08-10,A hack screenwriter writes a screenplay for a ...,8.4,en,"William Holden, Gloria Swanson, Erich von Stro...",Billy Wilder,Drama
2,548,Rashomon,1950-08-26,Brimming with action while incisively examinin...,8.1,ja,"Toshirō Mifune, Machiko Kyō, Takashi Shimura, ...",Akira Kurosawa,"Crime, Drama, Mystery"
3,705,All About Eve,1950-10-06,From the moment she glimpses her idol at the s...,8.1,en,"Bette Davis, Anne Baxter, George Sanders, Cele...",Joseph L. Mankiewicz,Drama
4,37292,Broken Arrow,1950-08-01,Indian scout Tom Jeffords is sent out to stem ...,6.7,en,"James Stewart, Jeff Chandler, Debra Paget, Bas...",Delmer Daves,Western


In [382]:
def remove_space_from_names(words):
    new_string = ""
    for i in words:
        if i == " ":
            new_string += ""
        else:
            new_string += i
    return new_string

In [383]:
df["Top_5_cast"] = df["Top_5_cast"].apply(remove_space_from_names)
df["Directors"] = df["Directors"].apply(remove_space_from_names)
df["Genre_names"] = df["Genre_names"].apply(remove_space_from_names)

In [384]:
df["tags"] = df["Movie_name"].apply(remove_space_from_names) + " " + df["Top_5_cast"] + " " + df["Directors"] + " " + df["Genre_names"]+ " "+ df["Overview"]

In [385]:
def split_and_fill(row):
    genres = row['Genre_names'].split(',')
    while len(genres) < 3:
        genres.append('Unknown')
    genres.sort()
    return pd.Series(genres[:3], index=['Genre 1', 'Genre 2', 'Genre 3'])

In [386]:
def split_and_fill_cast(row):
    genres = row['Top_5_cast'].split(',')
    while len(genres) < 3:
        genres.append('Unknown')
    genres.sort()
    return pd.Series(genres[:3], index=['Top_1', 'Top_2', 'Top_3'])

In [387]:
df[['Genre 1', 'Genre 2', 'Genre 3']] = df.apply(split_and_fill, axis=1)

#clustering

In [388]:
from sklearn.preprocessing import OneHotEncoder

In [389]:
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df[['Genre 1', 'Genre 2', 'Genre 3',"Original_language"]])

c:\Users\ppppr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [390]:
from sklearn.cluster import KMeans


num_clusters = 500  # Choose the number of clusters you want

kmeans = KMeans(n_clusters=num_clusters, random_state=0)
df['Cluster'] = kmeans.fit_predict(encoded_features)

c:\Users\ppppr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [391]:
new_df = df[["Movie_id","Movie_name","tags","Cluster","Genre 1","Genre 2","Genre 3","Original_language"]]

In [392]:
new_df.head()

,Movie_id,Movie_name,tags,Cluster,Genre 1,Genre 2,Genre 3,Original_language
0,11224,Cinderella,"Cinderella IleneWoods,EleanorAudley,VernaFelto...",31,Animation,Family,Fantasy,en
1,599,Sunset Boulevard,"SunsetBoulevard WilliamHolden,GloriaSwanson,Er...",1,Drama,Unknown,Unknown,en
2,548,Rashomon,"Rashomon ToshirōMifune,MachikoKyō,TakashiShimu...",0,Crime,Drama,Mystery,ja
3,705,All About Eve,"AllAboutEve BetteDavis,AnneBaxter,GeorgeSander...",1,Drama,Unknown,Unknown,en
4,37292,Broken Arrow,"BrokenArrow JamesStewart,JeffChandler,DebraPag...",29,Unknown,Unknown,Western,en


In [393]:
def find_similar_movies(new_data_point):
    new_data_encoded = encoder.transform([new_data_point])
    cluster_label = kmeans.predict(new_data_encoded)[0]
    similar_movies = df[df['Cluster'] == cluster_label]
    return similar_movies  # Returns a subset of Data set where the Cluster is same as data point

In [394]:
import pickle

In [395]:
def get_genres_for_movie(movie_id):
    row = new_df[new_df['Movie_id'] == movie_id]
    if not row.empty:
        genres = row[['Genre 1', 'Genre 2', 'Genre 3',"Original_language"]].values[0].tolist()
        return genres
    else:
        return None  # Return None if the movie ID is not found

In [396]:
from sklearn.feature_extraction.text import CountVectorizer

In [397]:
vectorizer = CountVectorizer(max_features = 80000,stop_words = "english")

In [398]:
vectorizer.fit(new_df["tags"])

CountVectorizer(max_features=80000, stop_words='english')

In [399]:
new_df["Movie_key"] = df["Movie_name"] + " " + 	df["Release_date"]

C:\Users\ppppr\AppData\Local\Temp\ipykernel_1288\2078571689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Movie_key"] = df["Movie_name"] + " " + 	df["Release_date"]


In [400]:
pickle.dump(new_df,open("data.pkl","wb"))

In [401]:
pickle.dump(encoder,open("encoder.pkl","wb"))

In [402]:
pickle.dump(kmeans,open("kmeans.pkl","wb"))

In [403]:
pickle.dump(vectorizer,open("vectorizer.pkl","wb"))

In [404]:
from sklearn.metrics.pairwise import cosine_similarity

In [405]:
import requests
from IPython.display import display, Image

TMDB_API_KEY = 'b2b3610b2d734655e8f50a74599f5a62'  # Replace with your TMDB API key
BASE_URL = 'https://api.themoviedb.org/3'
POSTER_BASE_URL = 'https://image.tmdb.org/t/p/w500'

def fetch_and_show_movie_poster(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={TMDB_API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        movie_data = response.json()
        poster_path = movie_data.get('poster_path')
        
        if poster_path:
            poster_url = f"{POSTER_BASE_URL}/{poster_path}"
            display(Image(url=poster_url))
        else:
            print("No poster available for this movie.")
    else:
        print("Error fetching movie data.")


In [429]:
def predict(movie_id):
    data = find_similar_movies(get_genres_for_movie(movie_id))
    movie = vectorizer.transform(new_df[new_df["Movie_id"] == movie_id]["tags"]).toarray()
    matrix = []
    for index, value in data["tags"].items():
        others = vectorizer.transform([value]).toarray()
        cosine = cosine_similarity(movie,others)
        matrix.append((cosine,index))
    sorted_list = sorted(matrix, key=lambda x: x[0][0][0], reverse=True)
    for i in range(1,6):
        fetch_and_show_movie_poster(new_df["Movie_id"].iloc[sorted_list[i][1]])
        print(new_df["Movie_name"].iloc[sorted_list[i][1]])


In [430]:
predict(24428)

c:\Users\ppppr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Avengers: Age of Ultron


Avengers: Infinity War


Captain America: Civil War


Allegiant


Avengers: Endgame
